# Quantum Simulator: Vaccine Site Selector


Covid-19 vaccines must be readily available in areas most affected by the pandemic. While setting up vaccination centers it is very important to select locations which can fulfill the demand from the affected areas. Covid Vaccination center locator selects optimal locations for the centers from a given set of locations satisfying demand while minimizing cost. The solution works using Quantum Optimization Algorithms to help design the network of centers from a given locations. As compared to classical methods of optimization, it uses Simulated Quantum Annealing to lower the distribution/operations cost and compute time.


### Prerequisite

The kernel comes pre-installed with the required packages. Else ensure to have the following Python Packages in your environment at minimum:

    - numpy
    - pandas
### Usage instructions
Usage Methodology for the algorithm:
1. The input has to be a .csv file with the content in columns titled 'location', 'point', 'opening_costs', 'equip_costs'
1. The file should follow 'utf-8' encoding.
1. The input can have a maximum of 200 demand points and 200 locations.
1. 'opening_costs' column should contain the opening costs, for each location, with 1st row having cost for 0th location, 2nd row having cost for 1st location and so on. 
1. 'equip_costs' column should contain cost of readying a center for vaccination, which is assumed to be constant for all lcoations.

General instructions for consuming the service on Sagemaker:
1. Access to AWS SageMaker and the model package
1. An S3 bucket to specify input/output
1. Role for AWS SageMaker to access input/output from S3


 ### Contents

1. [Importing libraries for runtime](#Importing-libraries-for-runtime)
1. [Model](#Model)
1. [Batch Transform](#Batch-Transform)
1. [Output](#Output)
1. [Endpoint](#Endpoint)

## Importing libraries for runtime

In [1]:
import pandas as pd
import boto3
import re

### Input Format
The input has to be a .csv file with the content in columns titled 

'location', 'point', 'opening_costs', 'equip_costs'.

Rows must be arranged in the manner as explained by following example. 

| location | point | opening_cost | equip_cost |
|----------|-------|--------------|------------|
| 0        | 30    | 4807         | 457        |
| 0        | 48    | 4396         |
| 1        | 11    | 4029         |            |
| 1        | 14    | 4671         |            |
| 1        | 19    | 4323         |            |
...
| 2        | 13    |
| 2        | 17    |
...

Center can be built at location 0 satifying demand points 30 and 48.

Center can be built at location 1 satifying demand points 11, 14 and 19.

Center can be built at location 2 satifying demand points 13 and 17.

Columns 'opening_costs' and 'equip_costs' are unrelated to first 2 columns. 

'opening_costs' contains costs to build Centers at locations 0 to 49 (for a problem with 50 locations).

'equip_costs' contains cost of readying a Center for vaccination.

<b> Note: 
    Input file should be of the form .csv and with 'utf-8' encoding. Ensure Content-Type is 'text/csv'
</b>

In [2]:
input_df  = pd.read_csv("input.csv")
input_df.head(10)

,location,point,opening_costs,equip_costs
0,0,4,4644.0,337.0
1,0,10,4339.0,NaN
2,0,12,4483.0,NaN
3,0,13,4479.0,NaN
4,0,24,4127.0,NaN
5,0,27,4804.0,NaN
6,0,43,4521.0,NaN
7,1,0,4536.0,NaN
8,1,3,4610.0,NaN
9,1,11,4645.0,NaN


## Model

The model package must be loaded for predicting the optimal locations for given set of demands.


In [ ]:
model_package_arn = 'Put your ARN here'

In [3]:
model_package_arn = 'arn:aws:sagemaker:us-east-2:786796469737:model-package/covid-location-v1-10-28'

In [4]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [5]:
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


## Batch Transform

In [6]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.large')
transformer.transform('s3://mphasis-marketplace/covid-location/input.csv', content_type='text/csv')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")
bucketFolder = transformer.output_path.rsplit('/')[3]

......................2020-10-28T22:18:47.303:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [28/Oct/2020 22:18:47] "#033GET /ping HTTP/1.1#033" 200 -
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [28/Oct/2020 22:18:47] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [28/Oct/2020 22:18:47] "#033GET /execution-parameters HTTP/1.1#033" 404 -
STEP 1 executed....
169.254.255.130 - - [28/Oct/2020 22:18:47] "#033GET /execution-parameters HTTP/1.1#033" 404 -
STEP 1 executed....
{'x(0)': 1, 'x(1)': 0, 'x(2)': 1, 'x(3)': 0, 'x(4)': 1, 'x(5)': 1, 'x(6)': 0, 'x(7)': 1, 'x(8)': 1, 'x


Batch Transform complete


In [7]:
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('FILE_NAME', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


## Output

- The processed output is of the form JSON file

- Output JSON file will contain list of all enabled locations for warehouses and net cost of the solution.

In [8]:
import json
f = open("FILE_NAME") 
data = json.load(f)
f.close()

print('\nEnabled Locations-> ', data['enabled_locations'])
print('\nNet Cost-> ', data['cost'])


Enabled Locations->  [0, 2, 4, 5, 7, 8, 10, 12, 15, 19, 23, 26, 27, 28, 30, 31, 32, 36, 37, 38, 41, 44, 45]

Net Cost->  208350.0


## Endpoint
Here is a sample endpoint for reference

In [9]:
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit

role = get_execution_role()

sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [10]:
content_type='text/csv'
model_name='COVIDLOCATION'
real_time_inference_instance_type='ml.m5.xlarge'

In [11]:
model_package_arn = 'Put your ARN here'

In [12]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [13]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [14]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

In [15]:
input_file_name = 'input.csv'
output_file_name = 'output.json'

In [16]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$input_file_name --content-type 'text/csv' --region us-east-2 $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


In [17]:
f = open(output_file_name,) 
data = json.load(f)
f.close() 

print('\nEnabled Locations-> ', data['enabled_locations'])
print('\nNet Cost-> ', data['cost'])


Enabled Locations->  [2, 3, 5, 7, 12, 13, 15, 16, 17, 20, 21, 22, 24, 25, 28, 29, 31, 32, 33, 35, 39, 40, 41, 42, 44, 47, 48, 49]

Net Cost->  211741.0


In [18]:
# Delete Endpoint
predictor.delete_endpoint()